```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Basic Particle Filter

In this notebook we will make test implementations of basic particle filters.

The aim is to find a decent implementation of the particles, which can be used by both simulators and particle filter.

All post-processing of particles will be done on the CPU in this first iteration.

## Overview of basic resampling strategies

The following four resampling algorithms are described in Section 3a) in van Leeuven's 2009 review paper.

The starting point is a prior distribution of model states with pdf $p(\psi^0)$, from which $N$ model state samples (particles) $\psi_i^0$, $i = 1,...,N$ are drawn.
Run the simulation model on all particles $\psi_i^n = f(\psi_i^{n-1})$. This is the same as sampling from the $p(\psi^n | \psi_i^{n-1})$.
At this point we see an observation $d$.

Now, define a posterior distribution $p(\psi^n | d)$. **Think a bit here before writing more** - from this we obtain weights $w_i$.


The most basic resampling strategies are as follows:
### Probabilistic resampling
Use the weights as a discrete distribution and sample directly from this.

### Residual sampling
Here, we first resample particles deterministic based on their weights. Resample particle $i$ np.floor$(Nw_i)$ times. Define the left-over weights as $w^*_i = Nw_i \% 1$, and use $w^*_i$ as a discrete distribution, from which the reminder resampled particles are drawn from, until we have $N$ particles again.

**Note:** It is here recommended to use Cauchy distribution as the observation pdf.

### Stochastic Universal sampling
Put all weights as buckets on the line $[0, 1]$, and draw a random number $u \sim U[0, 1/N]$.
Put $N$ line pieces starting from $u$ with length $1/N$ are laid on the line $[0,1]$. 
The bucket in which each line piece ends (not started???) defines which particle is chosen for each of the $N$ line pieces.

### Monte Carlo Metropolis-Hastings sampling
This sampling scheme is described very algorithmic in the paper already, so take a look at 3a4).


#### Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec


import os
import pyopencl
import datetime
import sys

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')




In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

## Thoughts on code structure

The observation will in these initial cases be a chosen model realization. When initializing a data assimilation with N particles, N+1 particles should be created and distributed on simulators.

One hypothesis for our ocean simulator is that integrating 100 particles within the same simulation is equally expensive as integrating 1 particle. Each particle integration should be done with a single thread on the GPU, so all 100 particles will can be processed in parallel.

If this assumption is true, it is best to have all particle positions continuous in memory. Hence, it will be implemented as an struct of array.

## Create random particles, and create random observation

In [ ]:
# Seed so that all simulation runs are equal.
np.random.seed(10)

print np.zeros((2,3))
print np.ones(4)
a = np.random.rand(3,2)
print a
print a.shape
print 

In [ ]:
### GlobalParticlesClass

class GlobalParticles:
    def __init__(self, numParticles):
        
        self.numParticles = numParticles
        
        # Observation index is the last particle
        self.obs_index = self.numParticles 
        self.observation_gamma = 0.1
        
        # One position for every particle plus observation
        self.positions = np.zeros((self.numParticles + 1, 2))
        
    def initializeUnitSquare(self):
        self.positions = np.random.rand(self.numParticles + 1, 2)
    
        # Ensure that the observation is in the middle 0.5x0.5 square:
        self.positions[self.obs_index, :] = self.positions[self.obs_index]*0.5 + 0.25
        
    def getDistances(self):
        distances = np.zeros(self.numParticles)
        for i in range(self.numParticles):
            distances[i] = np.sqrt( (self.positions[i,0]-self.positions[self.obs_index,0])**2 +
                                    (self.positions[i,1]-self.positions[self.obs_index,1])**2)
        return distances
        
    def getParticlePositions(self):
        return self.positions[:-1,:]
    
    def getObservationPosition(self):
        return self.positions[-1, :]
    
    def getGaussianWeight(self, distance=None, normalize=True):
        if distance is None:
            distance = self.getDistances()
        weights = (1.0/np.sqrt(2*np.pi*self.observation_gamma**2))* \
            np.exp(- (distance**2/(2*self.observation_gamma**2)))
        if normalize:
            return weights/np.sum(weights)
        return weights
    
    """
    Weights are calculated using a Cauchy Distribution.
    It is chosen over a Gauss distribution in order to obtain wider tails.
    """
    def getCauchyWeight(self, distance=None, normalize=True):
        if distance is None:
            distance = self.getDistances()
        weights = 1.0/(np.pi*self.observation_gamma*(1 + (distance/self.observation_gamma)**2))
        if normalize:
            return weights/np.sum(weights)
        return weights    
    
    def plotDistanceInfo(self, title=None):
        fig = plt.figure(figsize=(10,6))
        gridspec.GridSpec(2, 3)
        
        # PLOT POSITIONS OF PARTICLES AND OBSERVATIONS
        ax0 = plt.subplot2grid((2,3), (0,0))
        plt.plot(self.getParticlePositions()[:,0], \
                 self.getParticlePositions()[:,1], 'b.')
        plt.plot(self.getObservationPosition()[0], \
                 self.getObservationPosition()[1], 'r.')
        plt.xlim(0, 1)
        plt.xlabel('x')
        plt.ylabel('y')
        plt.ylim(0, 1)
        plt.title("Particle positions")
        
        # PLOT DISCTRIBUTION OF PARTICLE DISTANCES AND THEORETIC OBSERVATION PDF
        ax0 = plt.subplot2grid((2,3), (0,1), colspan=2)
        distances = self.getDistances()
        plt.hist(distances, bins=30, range=(0,1), normed=True, label="particle distances")
        
        # With observation 
        x = np.linspace(0, 1.0, num=100)
        cauchy_pdf = self.getCauchyWeight(x, normalize=False)
        gauss_pdf = self.getGaussianWeight(x, normalize=False)
        plt.plot(x, cauchy_pdf, 'r', label="obs Cauchy pdf")
        plt.plot(x, gauss_pdf, 'g', label="obs Gauss pdf")
        plt.legend()
        plt.title("Distribution of particle distances from observation")
        
        # PLOT SORTED DISTANCES FROM OBSERVATION
        ax0 = plt.subplot2grid((2,3), (1,0), colspan=3)
        cauchyWeights = self.getCauchyWeight(distances)
        gaussWeights = self.getGaussianWeight(distances)
        indices_sorted_by_observation = distances.argsort()
        plt.plot(distances[indices_sorted_by_observation], label="distance")
        plt.plot(cauchyWeights[indices_sorted_by_observation]/np.max(cauchyWeights), 'r', label="Cauchy weight")
        plt.plot(gaussWeights[indices_sorted_by_observation]/np.max(gaussWeights), 'g', label="Gauss weight")
        plt.title("Sorted distances from observation")
        plt.grid()
        plt.ylim(0,1.4)
        plt.legend()
        if title is not None:
            plt.suptitle(title)

# Initialize an ensemble of particles:
N = 1000
globalParticles = GlobalParticles(N)
globalParticles.initializeUnitSquare()

# The particles are by now drawn from the prior distribution

# Here, the simulation/time integration should take place

# Inspect initial ensemble
globalParticles.plotDistanceInfo(title="Initial particles")
print "Observation: ", globalParticles.getObservationPosition()

# Run particle filter




In [ ]:
globalParticles.plotDistanceInfo(title="Initial particles")

# Create a new GlobalParticles instance based on newSampleIndices
def resampleParticles(particles, newSampleIndices):
    newParticles = GlobalParticles(particles.numParticles)
    
    # Simply copy the given positions
    newParticles.positions[:-1,:] = particles.positions[newSampleIndices,:].copy()
    
    # Copy the observation:
    newParticles.positions[-1,:] = particles.positions[-1,:]
    
    return newParticles
    

def probabilisticResampling(particles):
    # Obtain weights:
    weights = particles.getGaussianWeight()
    #weights = particles.getCauchyWeight()
    
    # Draw new indices based from discrete distribution based on weights
    allIndices = range(particles.numParticles)
    newSampleIndices = np.random.choice(allIndices, particles.numParticles, p=weights)
        
    # Return a new set of particles
    return resampleParticles(particles, newSampleIndices)
 
    
    
    
probabilisticResampledParticles = probabilisticResampling(globalParticles)
probabilisticResampledParticles.plotDistanceInfo(title="From ProbabilisticResampling")



In [ ]:
#### Sorting example from https://stackoverflow.com/a/21077060

people = np.array(['Jim', 'Pam', 'Micheal', 'Dwight'])
ages = np.array([27, 25, 4, 9])
sorted_indices = ages.argsort()
print people[sorted_indices]

In [ ]:
a = 12312.12312
print a % 1
print np.floor(a)